In [151]:
import rapidfuzz
from rapidfuzz.process import extract, extract_iter , extractOne
import fuzzywuzzy
import pandas as pd
import itertools
import json

In [152]:
mapi = pd.read_csv("mapi_list_sn_ln.csv")
mapi.iloc[3752][0] = ""

In [153]:
73
answers = ['No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 '- Fullerton Advanced Balance Scale\n- Falls Efficacy Scale\n- SF-36\n- Geriatric Depression Scale\n- Geriatric Anxiety Scale\n- Apathy Evaluation Scale\n- Expanded Short Physical Performance Battery\n- Zarit Caregiver Burden Scale\n- Neuropsychiatric Inventory Questionnaire\n- Alternative Uses Task\n- Postural Sway\n- Gait Speed and Variability\n- Timed Up and Go\n- Philadelphia Mindfulness Scale\n- Abbreviated Cognitive Battery\n- Multisensory Reaction Time\n- MRI scan of brain (not a questionnaire)\n- IMOVE Protocol, Version 6.0 (not a questionnaire)',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.']

In [177]:
# 69
answers = ['- FACT-G (Version 4)\n- EQ-5D-5L Questionnaire',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 '- EQ-5D-5L, EuroQol group\n- FACT-G, Functional Assessment of Cancer Therapy-General',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 'No validated clinical questionnaires were used in this text.',
 '- FACT-G (Version 4)\n- EQ-5D-5L Questionnaire',
 '- EQ-5D-5L questionnaire\n- FACT-G questionnaire']

In [178]:
def key(tup):
    return tup[1]

In [179]:
def answer_cleaning(my_list):
    temp_list1 = []
    temp_list2 = []
    for x in my_list:
        # if not x == 'Not found.':
        if not x == 'No validated clinical questionnaires were used in this text.':
            temp_list1.append(x)
    for y in temp_list1:
        split = y.split('\n')
        for s in split:
            temp_list2.append(s)
    return temp_list2

In [180]:
def weighted_iter_long(string, long=True):
    matches = []
    choices = mapi.long_name
    if long == False:
        choices = mapi.short_name
    cut_off = 0.7

    for choice in choices:
        score = rapidfuzz.distance.Levenshtein.normalized_similarity(string, choice, processor=rapidfuzz.utils.default_process, weights=(1,0.999999,1))
        if score >= cut_off and "..." not in choice:
            matches.append((choice, score))
        
    matches = sorted(matches, key=key, reverse=True)
    
    return matches

In [181]:
# join algorithms to match name
def compound(string, long=True):

    weighted = weighted_iter_long(string,long)
    ratio = []
    for answer in weighted:
        y = rapidfuzz.fuzz.partial_ratio(string, answer[0])/100
        score = answer[1] + y
        ratio.append((answer[0], score))
    actual = []
    for answer in ratio:
        x = rapidfuzz.distance.Levenshtein.normalized_similarity(string, answer[0], weights=(1,1,1))
        score = answer[1] + x
        actual.append((answer[0], score))
    
    actual = sorted(actual, key=key, reverse=True)
    cutoff = 1.6
    if len(actual) > 1:
        if actual[1][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    elif len(actual) == 1:
        if actual[0][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    else:
        actual = ()
    return actual

In [182]:
# short + long names 
def joint(string):
    short = compound(string, long=False)
    long  = compound(string)
    best = (long, 'l')
    if short and not long:
        best = (short,'s')
    if short and long:
        if short[1] > long[1]:
            best = (short,'s')
    if best[0]:
        best = best[0][0], best[1]
    return best

In [183]:
def pre_cleaning_answers(my_list):
    new = list(filter(None, my_list))
    almost = {}
    short = []
    long = []
    for q in new:
        if q[1] == 's':
            short.append(q[0])
        else:
            long.append(q[0])
    short = list(filter(None, short))
    long = list(filter(None, long))
    for q in short:
        idx = mapi.index[mapi['short_name'] == q]
        if mapi.iloc[idx[0]][0] not in almost:
            almost[mapi.iloc[idx[0]][0]] = mapi.iloc[idx[0]][1]

    for q in long:
        idx = mapi.index[mapi['long_name'] == q]
        if mapi.iloc[idx[0]][0] not in almost:
            almost[mapi.iloc[idx[0]][0]] = mapi.iloc[idx[0]][1]


    return almost

In [184]:
def list_cleaning(my_list):
    temp_list = []
    my_list = list(filter(None, my_list))
    for token in my_list:
        temp_list.append(rapidfuzz.utils.default_process(token))
        temp_list.append(token)
    temp_list = list(dict.fromkeys(my_list))

    return temp_list

In [185]:
cleaned_answers = answer_cleaning(answers)

In [186]:
cleaned_answers

['- FACT-G (Version 4)',
 '- EQ-5D-5L Questionnaire',
 '- EQ-5D-5L, EuroQol group',
 '- FACT-G, Functional Assessment of Cancer Therapy-General',
 '- FACT-G (Version 4)',
 '- EQ-5D-5L Questionnaire',
 '- EQ-5D-5L questionnaire',
 '- FACT-G questionnaire']

In [187]:
x = []
for answer in cleaned_answers:
    x.append(joint(answer))

In [188]:
x

[('FACT-G', 's'),
 ('EAR-Questionnaire', 'l'),
 ('EQ-5D-5L', 's'),
 ('Functional Assessment of Cancer Therapy - General', 'l'),
 ('FACT-G', 's'),
 ('EAR-Questionnaire', 'l'),
 ('EQ-5D-5L', 's'),
 ('ChAt questionnaire', 's')]

In [189]:
y = pre_cleaning_answers(x)

In [190]:
y

{'FACT-G': 'Functional Assessment of Cancer Therapy - General',
 'EQ-5D-5L': 'EuroQoL 5-Dimension 5-Level',
 'ChAt questionnaire': 'Children Atopy questionnaire',
 'EAR-Q': 'EAR-Questionnaire'}